In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor  # Meta-model
from sklearn.metrics import mean_absolute_error, mean_squared_error

# Step 1: Load Predictions from Base Models
lstm_preds = pd.read_csv("lstm_preds.csv", header=None, names=["LSTM"])
xgboost_preds = pd.read_csv("xgboost_preds.csv", header=None, names=["XGBoost"])
lightgbm_preds = pd.read_csv("lightgbm_preds.csv", header=None, names=["LightGBM"])
svr_preds = pd.read_csv("svr_preds.csv", header=None, names=["SVR"])
rf_preds = pd.read_csv("random_forest_preds.csv", header=None, names=["RandomForest"])

# Load actual validation set target values
y_validation = pd.read_csv("y_validation.csv", header=None, names=["Actual"])

# Step 2: Merge Predictions into a Single DataFrame
meta_features = pd.concat([lstm_preds, xgboost_preds, lightgbm_preds, svr_preds, rf_preds], axis=1)
meta_target = y_validation["Actual"]

# Step 3: Split Data for Training Meta-Model
X_train_meta, X_val_meta, y_train_meta, y_val_meta = train_test_split(meta_features, meta_target, test_size=0.2, random_state=42)

# Step 4: Train the Meta-Model (Gradient Boosting Regressor)
meta_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
meta_model.fit(X_train_meta, y_train_meta)

# Step 5: Predict on Validation Set and Evaluate
meta_preds = meta_model.predict(X_val_meta)

mae = mean_absolute_error(y_val_meta, meta_preds)
rmse = np.sqrt(mean_squared_error(y_val_meta, meta_preds))

print(f"Meta-Model MAE: {mae}")
print(f"Meta-Model RMSE: {rmse}")

# Step 6: Use the Meta-Model for Final Predictions (on Test Set)
# Load test set predictions from base models
lstm_test_preds = pd.read_csv("lstm_test_preds.csv", header=None, names=["LSTM"])
xgboost_test_preds = pd.read_csv("xgboost_test_preds.csv", header=None, names=["XGBoost"])
lightgbm_test_preds = pd.read_csv("lightgbm_test_preds.csv", header=None, names=["LightGBM"])
svr_test_preds = pd.read_csv("svr_test_preds.csv", header=None, names=["SVR"])
rf_test_preds = pd.read_csv("random_forest_test_preds.csv", header=None, names=["RandomForest"])

# Combine test predictions
test_meta_features = pd.concat([lstm_test_preds, xgboost_test_preds, lightgbm_test_preds, svr_test_preds, rf_test_preds], axis=1)

# Generate final ensemble predictions
final_predictions = meta_model.predict(test_meta_features)

# Save final predictions
pd.DataFrame(final_predictions, columns=["Final_Predictions"]).to_csv("final_ensemble_predictions.csv", index=False)

print("Final ensemble predictions saved as 'final_ensemble_predictions.csv'")